# Gorgias Ticket Analysis

This notebook provides a lightweight client to fetch tickets and conversations from the Gorgias API for analysis.

## 1. Import Required Libraries

In [2]:
import httpx
import time
import os
from typing import Any, Dict, List, Optional
from datetime import datetime, timezone
from dotenv import load_dotenv

## 2. Configure Gorgias API Credentials

Set your Gorgias credentials below. You can either:
- Set environment variables: `GORGIAS_DOMAIN`, `GORGIAS_USERNAME`, `GORGIAS_API_KEY`
- Or replace the values directly in the config dict

In [12]:
# Load env vars
load_dotenv()

# Configuration - set your credentials here or use environment variables
config = {
    "domain": os.environ.get("GORGIAS_DOMAIN"),
    "username": os.environ.get("GORGIAS_USERNAME"),
    "api_key": os.environ.get("GORGIAS_API_KEY"),
}

# Validate elke key value pair, loopt door elke key value en pakt de key als de waarde None is.
missing = [key for key, value in config.items() if value is None]
if missing:
    raise ValueError(f"Missing environment variables: {', '.join(missing)}")

print(f"Configuration loaded. Domain: {config['domain']}, Username: {config['username']}")

Configuration loaded. Domain: northenoak.gorgias.com, Username: admin@northenoak.com


## 3. Gorgias API Client

A lightweight HTTP client with:
- Basic authentication
- Rate limiting (40 requests per 20 seconds)
- Retry logic with exponential backoff
- Cursor-based pagination

In [13]:
class GorgiasClient:
    """
    HTTP client for Gorgias API with rate limiting.
    
    Gorgias Rate Limits (API key): 40 requests per 20 seconds.
    """
    
    def __init__(
        self,
        domain: str,
        username: str,
        api_key: str,
        request_delay: float = 0.6
    ):
        """
        Initialize Gorgias client.
        
        Args:
            domain: Gorgias domain (e.g., 'mycompany.gorgias.com')
            username: API username (email)
            api_key: API key
            request_delay: Delay between requests in seconds (default 0.6s = ~33 req/20s)
        """
        self.base_url = f"https://{domain}/api"
        self.auth = httpx.BasicAuth(username, api_key)
        self._request_count = 0
        self._last_request_time = None
        self._request_delay = request_delay
        self._client = None
    
    def _get_client(self) -> httpx.Client:
        """Get or create HTTP client."""
        if self._client is None or self._client.is_closed:
            self._client = httpx.Client(
                base_url=self.base_url,
                auth=self.auth,
                timeout=60.0
            )
        return self._client
    
    def close(self):
        """Close the HTTP client."""
        if self._client and not self._client.is_closed:
            self._client.close()
    
    def _throttle(self):
        """Apply rate limiting delay."""
        if self._last_request_time:
            elapsed = (datetime.now(timezone.utc) - self._last_request_time).total_seconds()
            if elapsed < self._request_delay:
                time.sleep(self._request_delay - elapsed)
    
    def request(
        self,
        method: str,
        endpoint: str,
        params: dict = None,
        max_retries: int = 5
    ) -> Any:
        """
        Make HTTP request with rate limit handling.
        
        Args:
            method: HTTP method (GET, POST, etc.)
            endpoint: API endpoint (e.g., 'tickets', 'tickets/123')
            params: Query parameters
            max_retries: Maximum retry attempts
            
        Returns:
            Response JSON data
        """
        client = self._get_client()
        
        for attempt in range(max_retries + 1):
            self._throttle()
            self._last_request_time = datetime.now(timezone.utc)
            
            try:
                response = client.request(
                    method, f"/{endpoint.lstrip('/')}", params=params
                )
                self._request_count += 1
                
                # Handle rate limiting (429)
                if response.status_code == 429:
                    retry_after = min(int(response.headers.get("Retry-After", 2 ** attempt)), 60)
                    print(f"Rate limited. Waiting {retry_after}s...")
                    time.sleep(retry_after)
                    continue
                
                response.raise_for_status()
                return response.json() if response.status_code != 204 else None
                
            except Exception as e:
                if attempt == max_retries:
                    raise Exception(f"Gorgias API request failed: {e}")
                wait_time = 2 ** attempt
                print(f"Request failed, retrying in {wait_time}s... ({e})")
                time.sleep(wait_time)
        
        raise Exception(f"Gorgias API failed after {max_retries} retries")
    
    def paginate(
        self,
        endpoint: str,
        params: dict = None,
        limit: int = 100,
        max_pages: int = None
    ) -> List[Dict]:
        """
        Fetch all pages from a paginated endpoint.
        
        Args:
            endpoint: API endpoint
            params: Additional query parameters
            limit: Page size (max 100)
            max_pages: Optional limit on number of pages to fetch
            
        Returns:
            List of all items from all pages
        """
        all_items = []
        cursor = None
        params = {**(params or {}), "limit": limit}
        page_count = 0
        
        while True:
            if cursor:
                params["cursor"] = cursor
            
            data = self.request("GET", endpoint, params=params)
            items = data.get("data", [])
            all_items.extend(items)
            page_count += 1
            
            print(f"  Fetched page {page_count}: {len(items)} items (total: {len(all_items)})")
            
            cursor = data.get("meta", {}).get("next_cursor")
            if not cursor:
                break
            
            if max_pages and page_count >= max_pages:
                print(f"  Stopped at max_pages={max_pages}")
                break
        
        return all_items
    
    @property
    def request_count(self) -> int:
        """Get total request count."""
        return self._request_count


# Create client instance
client = GorgiasClient(
    domain=config["domain"],
    username=config["username"],
    api_key=config["api_key"]
)
print(f"✓ Gorgias client initialized for {config['domain']}")

✓ Gorgias client initialized for northenoak.gorgias.com


## 4. Fetch Tickets

Fetch tickets from the Gorgias API. You can customize:
- `max_pages`: Limit the number of pages to fetch (for testing)
- `order_by`: Sort order (default: newest first)

In [14]:
# Fetch tickets (limit to 2 pages for testing - remove max_pages for all tickets)
print("Fetching tickets...")
tickets = client.paginate(
    endpoint="tickets",
    params={"order_by": "created_datetime:desc"},
    limit=100,
    max_pages=2  # Remove this line to fetch ALL tickets
)

print(f"\n✓ Fetched {len(tickets)} tickets")
print(f"  API calls made: {client.request_count}")

Fetching tickets...
  Fetched page 1: 100 items (total: 100)
  Fetched page 2: 100 items (total: 200)
  Stopped at max_pages=2

✓ Fetched 200 tickets
  API calls made: 2


## 5. Fetch Ticket Details with Messages

The ticket list only contains summary data. To get the full conversation (messages), 
fetch each ticket's details individually.

In [15]:
def fetch_ticket_details(client: GorgiasClient, ticket_ids: List[int], max_tickets: int = None) -> List[Dict]:
    """
    Fetch full ticket details including messages for each ticket.
    
    Args:
        client: GorgiasClient instance
        ticket_ids: List of ticket IDs to fetch
        max_tickets: Optional limit on number of tickets to fetch details for
        
    Returns:
        List of ticket details with messages
    """
    detailed_tickets = []
    ids_to_fetch = ticket_ids[:max_tickets] if max_tickets else ticket_ids
    
    print(f"Fetching details for {len(ids_to_fetch)} tickets...")
    
    for i, tid in enumerate(ids_to_fetch):
        try:
            ticket = client.request("GET", f"tickets/{tid}")
            detailed_tickets.append(ticket)
            
            if (i + 1) % 10 == 0:
                print(f"  Progress: {i + 1}/{len(ids_to_fetch)} tickets")
                
        except Exception as e:
            print(f"  Error fetching ticket {tid}: {e}")
    
    print(f"✓ Fetched details for {len(detailed_tickets)} tickets")
    return detailed_tickets


# Get ticket IDs from the list
ticket_ids = [t["id"] for t in tickets if t.get("id")]

# Fetch details for first 10 tickets (for testing - remove max_tickets for all)
detailed_tickets = fetch_ticket_details(client, ticket_ids, max_tickets=10)

Fetching details for 10 tickets...
  Progress: 10/10 tickets
✓ Fetched details for 10 tickets


## 6. Explore Ticket Structure

Let's look at the structure of a ticket with its messages.

In [ ]:
# Look at the first ticket's structure
if detailed_tickets:
    sample = detailed_tickets[5]
    print("Ticket keys:", list(sample.keys()))
    print(f"\nTicket ID: {sample.get('id')}")
    print(f"Status: {sample.get('status')}")
    print(f"Messages tag: {sample.get('tags')}")
    print(f"Ticket custom_fields: {sample.get('custom_fields')}")
    
# Ticket id 37499908
# Auto close is tag is gerelateerd aan spam, no-reply emails etc.
# Ticket id 37491924
# ORDER-STATUS tag is gerelateerd aan order status updates, AI classificatie vanuit Gorgias




Ticket keys: ['id', 'uri', 'external_id', 'events', 'status', 'priority', 'channel', 'via', 'from_agent', 'spam', 'customer', 'assignee_user', 'assignee_user_id', 'assignee_team', 'assignee_team_id', 'language', 'subject', 'summary', 'meta', 'tags', 'custom_fields', 'messages', 'created_datetime', 'opened_datetime', 'last_received_message_datetime', 'last_message_datetime', 'updated_datetime', 'closed_datetime', 'trashed_datetime', 'snooze_datetime', 'satisfaction_survey', 'reply_options', 'requester', 'is_unread']

Ticket ID: 37491924
Status: open
Messages tag: [{'id': 148717, 'name': 'ORDER-STATUS', 'decoration': {'color': '#84db2d'}}]
Ticket custom_fields: {'21198': {'id': 21198, 'value': 'Order::Status::Other'}}


## 7. Extract Conversations

Extract the message threads from tickets into a flat structure for analysis.

In [40]:
def extract_conversations(tickets: List[Dict]) -> List[Dict]:
    """
    Extract messages from tickets into a flat list for analysis.
    
    Args:
        tickets: List of ticket dicts with messages
        
    Returns:
        List of message records with ticket context
    """
    conversations = []
    
    for ticket in tickets:
        ticket_id = ticket.get("id")
        ticket_status = ticket.get("status")
        ticket_tag = ticket.get("tags")
        ticket_custom = ticket.get("custom_fields")
        customer = ticket.get("customer") or {}
        
        for msg in ticket.get("messages", []):
            sender = msg.get("sender") or {}
            
            conversations.append({
                "ticket_id": ticket_id,
                "ticket_tag": ticket_tag,
                "ticket_custom": ticket_custom,
                "ticket_status": ticket_status,
                "customer_name": customer,
                "body_text": msg.get("body_text"),
                "created_datetime": msg.get("created_datetime"),
            })
    
    return conversations


# Extract all conversations
conversations = extract_conversations(detailed_tickets)
print(f"✓ Extracted {len(conversations)} messages from {len(detailed_tickets)} tickets")

✓ Extracted 10 messages from 10 tickets


## 8. Analyze with Pandas (Optional)

Convert to a pandas DataFrame for easier analysis.

In [41]:
import pandas as pd

# Create DataFrames
df_conversations = pd.DataFrame(conversations)

# Show conversation messages preview
print("Conversations preview:")
display(df_conversations)

Conversations preview:


,ticket_id,ticket_tag,ticket_custom,ticket_status,customer_name,body_text,created_datetime
0,37499908,"[{'id': 148734, 'name': 'auto-close', 'decorat...","{'21198': {'id': 21198, 'value': 'Other::No Re...",closed,"{'id': 292537433, 'email': 'reply-6176270-114_...",(https://www.postnl.nl/favicon-postnl-32.png) ...,2026-01-17T12:40:21+00:00
1,37493366,[],None,open,"{'id': 292402332, 'email': 'syngameheraecsiots...",Meta Business Platform Onmiddellijke actie ver...,2026-01-17T11:07:10+00:00
2,37492958,"[{'id': 149315, 'name': 'urgent', 'decoration'...","{'21198': {'id': 21198, 'value': 'Exchange::Re...",open,"{'id': 282618106, 'email': 'bvw2301@gmail.com'...",Hallo \n\nIk heb vandaag mijn pakketje binnen ...,2026-01-17T11:01:19+00:00
3,37492790,"[{'id': 148755, 'name': 'feedback', 'decoratio...","{'21198': {'id': 21198, 'value': 'Order::Refun...",open,"{'id': 292391376, 'email': 'betalingen.lagerwe...","Geachte heer/mevrouw, \n \nOp 2 januari jl. he...",2026-01-17T10:59:44+00:00
4,37492769,"[{'id': 148717, 'name': 'ORDER-STATUS', 'decor...","{'21198': {'id': 21198, 'value': 'Order::Statu...",open,"{'id': 289093463, 'email': 'zellmanncor@gmail....",Goedendag \n\nIk heb bij jullie 14 januari wat...,2026-01-17T10:59:34+00:00
5,37491924,"[{'id': 148717, 'name': 'ORDER-STATUS', 'decor...","{'21198': {'id': 21198, 'value': 'Order::Statu...",open,"{'id': 267030971, 'email': 'mvernoist03@gmail....",Je zal het niet geloven....\r\nPaket net ontva...,2026-01-17T10:46:19+00:00
6,37491775,[],None,open,"{'id': 292368617, 'email': 'aiurdzkaslcsakrmat...",Urgent Notification: Meta Policy Enforcement \...,2026-01-17T10:44:14+00:00
7,37491417,"[{'id': 148717, 'name': 'ORDER-STATUS', 'decor...","{'21198': {'id': 21198, 'value': 'Order::Statu...",open,"{'id': 267030971, 'email': 'mvernoist03@gmail....","Goedemorgen, \nHelaas heb ik mijn bestelling n...",2026-01-17T10:37:52+00:00
8,37490115,"[{'id': 148722, 'name': 'RETURN/EXCHANGE', 'de...","{'21198': {'id': 21198, 'value': 'Return::Requ...",open,"{'id': 271907187, 'email': 'wouter@phimac.be',...",Beste \n \nIk wens de bestelling met nummer #1...,2026-01-17T10:18:24+00:00
9,37489589,"[{'id': 149315, 'name': 'urgent', 'decoration'...","{'21198': {'id': 21198, 'value': 'Other::No Re...",open,"{'id': 57436295, 'email': 'noreply@dhlecommerc...",Binnenkort staat onze bezorger met zending JVG...,2026-01-17T10:11:04+00:00


In [ ]:
# Show conversation messages preview
print("Conversations preview:")
df_conversations[["ticket_id", "sender_type", "sender_name", "body_text"]].head(10) 

Conversations preview:


,ticket_id,sender_type,sender_name,body_text
0,37499908,email,PostNL,(https://www.postnl.nl/favicon-postnl-32.png) ...
1,37493366,email,Meta For Business,Meta Business Platform Onmiddellijke actie ver...
2,37492958,email,Bianca van Winkel,Hallo \n\nIk heb vandaag mijn pakketje binnen ...
3,37492790,email,Martin Lagerweij,"Geachte heer/mevrouw, \n \nOp 2 januari jl. he..."
4,37492769,email,Cor Zellmann,Goedendag \n\nIk heb bij jullie 14 januari wat...
5,37491924,email,Marja Lelieveld,Je zal het niet geloven....\r\nPaket net ontva...
6,37491775,email,Meta For Business,Urgent Notification: Meta Policy Enforcement \...
7,37491417,email,Marja Lelieveld,"Goedemorgen, \nHelaas heb ik mijn bestelling n..."
8,37490115,email,Wouter D'hont,Beste \n \nIk wens de bestelling met nummer #1...
9,37489589,email,DHL eCommerce,Binnenkort staat onze bezorger met zending JVG...


## 9. Cleanup

Close the HTTP client when done.

In [ ]:
# Close the client connection
client.close()
print(f"✓ Client closed. Total API calls: {client.request_count}")